<a href="https://colab.research.google.com/github/gastan81/generative_ai/blob/main/0_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot

## 1. Settings

Installation

In [1]:
%%bash
pip install -qqq -U faiss-cpu
#!pip install -qqq -U langchain
pip install -qqq -U langchain-community
pip install -qqq -U langchain-huggingface
pip install -qqq -U pypdf
pip install -qqq -U streamlit

Libraries

In [9]:
import os
#from google.colab import userdata
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

Colab: token

In [3]:
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

Local: token

In [4]:
# token = os.getenv('HUGGINGFACEHUB_API_TOKEN')

Model

In [5]:
# hf_model = 'mistralai/Mistral-7B-Instruct-v0.3'
hf_model = 'microsoft/Phi-3.5-mini-instruct'
llm = HuggingFaceEndpoint(repo_id=hf_model)

C:\Users\Gastan\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


This will not be needed when everything on github:

https://drive.google.com/file/d/14PqeK1VNokE5PZ-b-hVLqV-h4xnSR7B4/view?usp=sharing Alice in Wonderland

https://drive.google.com/file/d/1mJIwux2e0XvZUDBoEj9rb3OlHlYsTxXb/view?usp=sharing Statistics in Python

https://drive.google.com/file/d/1i5ZhAxtIown7RzH1Sh13y2eqcXKvsMyq/view?usp=sharing Statistics in R

Embedding

In [6]:
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model
    , cache_folder=embeddings_folder)

Create vector from loaded document

In [ ]:
# # The document
# # # file_id = '1mJIwux2e0XvZUDBoEj9rb3OlHlYsTxXb' # Google drive file's ID
# # # file = 'https://drive.google.com/uc?export=download&id=' + file_id
# file = 'data/The CIA World Factbook 2018-2019.pdf'

In [ ]:
# # Read pages
# loader = PyPDFLoader(file)
# pages = []
# async for page in loader.alazy_load():
#     pages.append(page)

In [11]:
# # Split text
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
# docs = text_splitter.split_documents(pages)

In [ ]:
# # Review the loaded document
# print(f"{docs[10].metadata}\n")
# print(docs[10].page_content)

In [ ]:
vector_db = FAISS.from_documents(docs, embeddings)

In [11]:
vector_db.save_local("data/CIA_faiss_index")

Load vector from saved index

In [13]:
vector_db = FAISS.load_local("data/CIA_faiss_index", embeddings, allow_dangerous_deserialization=True)

Retriever

In [14]:
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

Chat setup

In [15]:
template = """You are a nice chatbot having a conversation with a human. Answer the question based only on the following context and previous conversation. Keep your answers short, succinct and informative, so that the couterpart can learn from you.

Previous conversation:
{chat_history}

Context to answer question:
{context}

New human question: {input}
Response:
"""

# chat_history = []

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

doc_retriever = create_history_aware_retriever(
    llm, retriever, prompt
)

doc_chain = create_stuff_documents_chain(llm, prompt)

# rag_bot = create_retrieval_chain(
#     doc_retriever, doc_chain
# )

Chatbot

In [17]:
chain = create_retrieval_chain(
    doc_retriever, doc_chain
)
history = []
# Start the conversation loop
while True:
  user_input = input("\nYou: ")

  # Check for exit condition
  if user_input.lower() == 'end':
      print("Ending the conversation. Goodbye!")
      break

  # Get the response from the conversation chain
  response = chain.invoke({"input":user_input, "chat_history": history, "context": retriever})
  history.extend([{"role": "human", "content": response["input"]},{"role": "assistant", "content":response["answer"]}])
  # Print the chatbot's response
  print(response["answer"])


Assistant: 4.4% (2017 est.)




Response: Germany celebrates several festivals throughout the year, but the most widely recognized include:

1. Oktoberfest: Held in Munich, this famous beer festival runs from late September to the first weekend in October.
2. Christmas (Christkindlesmarkt): A festive market and fair that takes place in many German cities, with the best-known Christmas markets in Nuremberg, Dresden, and Heidelberg.
3. Carnival (Fasching): Predominantly celebrated in southern Germany, particularly in Cologne, Mainz, and Munich, Fasching is a time of parades, costumes, and street parties leading up to Lent.
4. Easter (Osterspaziergang): Easter is a significant religious festival celebrated with church services, processions, and traditional foods like Ostersuppe (Easter soup).
5. Walpurgis Night (Valborg): Celebrated on the last Friday before May Day (April 30), this spring festival is marked by bonfires, dancing, and singing to ward off evil spirits.
6. New Year's Eve (S

Streamlit

app.py

In [49]:
%%writefile rag_app.py

from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
import streamlit as st

# llm
hf_model = "mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceEndpoint(repo_id=hf_model)

# embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)

# load Vector Database
# allow_dangerous_deserialization is needed. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine
vector_db = FAISS.load_local("/content/faiss_index", embeddings, allow_dangerous_deserialization=True)

# retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

# prompt
template = """You are a nice chatbot having a conversation with a human. Answer the question based only on the following context and previous conversation. Keep your answers short and succinct.

Previous conversation:
{chat_history}

Context to answer question:
{context}

New human question: {input}
Response:"""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

# bot with memory
@st.cache_resource
def init_bot():
    doc_retriever = create_history_aware_retriever(llm, retriever, prompt)
    doc_chain = create_stuff_documents_chain(llm, prompt)
    return create_retrieval_chain(doc_retriever, doc_chain)

rag_bot = init_bot()


##### streamlit #####

st.title("Chatier & chatier: conversations in Wonderland")

# Initialise chat history
# Chat history saves the previous messages to be displayed
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# React to user input
if prompt := st.chat_input("Curious minds wanted!"):

    # Display user message in chat message container
    st.chat_message("human").markdown(prompt)

    # Add user message to chat history
    st.session_state.messages.append({"role": "human", "content": prompt})

    # Begin spinner before answering question so it's there for the duration
    with st.spinner("Going down the rabbithole for answers..."):

        # send question to chain to get answer
        answer = rag_bot.invoke({"input": prompt, "chat_history": st.session_state.messages, "context": retriever})

        # extract answer from dictionary returned by chain
        response = answer["answer"]

        # Display chatbot response in chat message container
        with st.chat_message("assistant"):
            st.markdown(response)

        # Add assistant response to chat history
        st.session_state.messages.append({"role": "assistant", "content":  response})

Writing rag_app.py


In [17]:
%%writefile data/streamlit_app.py

from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import streamlit as st

# llm
hf_model = 'microsoft/Phi-3.5-mini-instruct' # 'mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceEndpoint(repo_id=hf_model)

# embeddings
embedding_model = 'sentence-transformers/all-MiniLM-l6-v2'
embeddings_folder = '/content/'

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model
    , cache_folder=embeddings_folder)

# load Vector Database
# allow_dangerous_deserialization is needed. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine
vector_db = FAISS.load_local('data/CIA_faiss_index', embeddings, allow_dangerous_deserialization=True)

# retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

# prompt
template = """You are a nice chatbot having a conversation with a human.
Answer the question based only on the following context and previous conversation.
Keep your answers short, succinct and informative, so that the couterpart can learn from you.

Previous conversation:
{chat_history}

Context to answer question:
{context}

New human question: {input}
Response:"""

prompt = ChatPromptTemplate.from_messages([
    ('system', template),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
])

# bot with memory
@st.cache_resource
def init_bot():
    doc_retriever = create_history_aware_retriever(llm, retriever, prompt)
    doc_chain = create_stuff_documents_chain(llm, prompt)
    return create_retrieval_chain(doc_retriever, doc_chain)

rag_bot = init_bot()


##### streamlit #####

st.title('CIA World Factbook 2018-2019')

# Initialise chat history
# Chat history saves the previous messages to be displayed
if 'messages' not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message['role']):
        st.markdown(message['content'])

# React to user input
if prompt := st.chat_input('Curious minds wanted!'):

    # Display user message in chat message container
    st.chat_message('human').markdown(prompt)

    # Add user message to chat history
    st.session_state.messages.append({'role': 'human', 'content': prompt})

    # Begin spinner before answering question so it's there for the duration
    with st.spinner('Asking CIA...'):

        # send question to chain to get answer
        answer = rag_bot.invoke({'input': prompt, 'chat_history': st.session_state.messages, 'context': retriever})

        # extract answer from dictionary returned by chain
        response = answer['answer']

        # Display chatbot response in chat message container
        with st.chat_message('assistant'):
            st.markdown(response)

        # Add assistant response to chat history
        st.session_state.messages.append({'role': 'assistant', 'content':  response})

Overwriting data/streamlit_app.py


In [ ]:
def tunnel_prep():
    for f in ('cloudflared-linux-amd64', 'logs.txt', 'nohup.out'):
        try:
            os.remove(f'/content/{f}')
            print(f"Deleted {f}")
        except FileNotFoundError:
            continue

    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -q
    !chmod +x cloudflared-linux-amd64
    !nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &
    url = ""
    while not url:
        time.sleep(1)
        result = !grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1
        if result:
            url = result[0]
    return display(HTML(f'Your tunnel URL <a href="{url}" target="_blank">{url}</a>'))